## Setup

In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

## Input data directory
data_dir = "graph"
inputdirectory = Path(f"./data_input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## Load Documents

In [2]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
loader = PyPDFLoader("data_input/Arrete_1692_BO_7222_Fr_Dématérialisation.pdf")
# loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)


Number of chunks =  49
Nº 7222 – 30 moharrem 1445 (17-8-2023) BULLETIN OFFICIEL 1775 
 
 
– les conditions et modalités de recours et de mise en œuvre des enchères électroniques inversées ; 
– les conditions et modalités de l’achat sur des bons de commande par voie électronique ; 
– les conditions et modalités de dématérialisation  du nantissement des marchés publics ; 
– les conditions et modalités d’interopérabilité  avec les systèmes tiers ; 
– les conditions et modalités de dématérialisation  des documents et pièces ; 
– les modalités de dématérialisation  de l’étude préalable des projets de dossiers de consultations ;  
– les modalités de tenue et d’exploitation de la base de données des prestataires. 
ART. 2. – Au sens du présent arrêté, on entend par : 
a) portail des marchés publics : la plateforme nationale de dématérialisation des procédures de passation des marchés  publics 
dont la gestion est assurée par la Trésorerie générale  du Royaume ; 
b) acteurs du portail des march

## Create a dataframe of all the chunks

In [ ]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages[5:])
print(df.shape)
df.head()

(49, 4)


,text,source,page,chunk_id
0,1774 BULLETIN OFFICIEL Nº 7222 – 30 moharrem 1...,data_input/Arrete_1692_BO_7222_Fr_Dématérialis...,0,becca813af1e441e962a57832929541b
1,soumissionnaires et adjudicataires de marchés...,data_input/Arrete_1692_BO_7222_Fr_Dématérialis...,0,195ee3917d684599b706ed184b71f7ce
2,– les modalités de publication des information...,data_input/Arrete_1692_BO_7222_Fr_Dématérialis...,0,da9c908d14fd41f8a07ce9f3bddfbaea
3,Nº 7222 – 30 moharrem 1445 (17-8-2023) BULLETI...,data_input/Arrete_1692_BO_7222_Fr_Dématérialis...,1,31d6b3aad5a2413bb77fc56e97d79e0e
4,"concurrents notamment, les commissions d’appel...",data_input/Arrete_1692_BO_7222_Fr_Dématérialis...,1,6d589375fb2d4d628dc59d9709629102


## Extract Concepts

In [4]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [ ]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model='llama3.3')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph_llama3.3.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks_llama3.3.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph_llama3.3.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

[
   {
       "node_1": "Arrêté du ministre délégué",
       "node_2": "Dématérialisation des procédures",
       "edge": "est relatif à"
   },
   {
       "node_1": "Loi n° 112-13",
       "node_2": "Nantissement des marchés publics",
       "edge": "porte sur"
   },
   {
       "node_1": "Loi n° 43-20",
       "node_2": "Services de confiance pour les transactions électroniques",
       "edge": "réglemente"
   },
   {
       "node_1": "Loi n° 53-05",
       "node_2": "Échange électronique des données juridiques",
       "edge": "concerne"
   },
   {
       "node_1": "Loi n° 55-19",
       "node_2": "Simplification des procédures et formalités administratives",
       "edge": "vise à"
   },
   {
       "node_1": "Loi n° 69-00",
       "node_2": "Contrôle financier de l’Etat sur les entreprises publiques",
       "edge": "régit"
   },
   {
       "node_1": "Dahir n° 1-56-211",
       "node_2": "Garanties pécuniaires des soumissionnaires et adjudicataires de marchés publics",
       "ed

In [6]:
concepts_list

[{'node_1': 'Arrêté du ministre délégué',
  'node_2': "ministre de l'économie et des finances, chargé du budget",
  'edge': ' Publié par',
  'chunk_id': '3b43557b46c442bfad59617aad10dbc5'},
 {'node_1': 'Arrêté n° 1692-23',
  'node_2': '4 hija 1444 (23 juin 2023)',
  'edge': "Date d'émission",
  'chunk_id': '3b43557b46c442bfad59617aad10dbc5'},
 {'node_1': 'Arrêté',
  'node_2': 'dématérialisation des procédures, documents et pièces relatifs aux marchés publics',
  'edge': 'Portée',
  'chunk_id': '3b43557b46c442bfad59617aad10dbc5'},
 {'node_1': 'Loi n° 112-13',
  'node_2': 'nantissement des marchés publics',
  'edge': 'Rappelant',
  'chunk_id': '3b43557b46c442bfad59617aad10dbc5'},
 {'node_1': 'Loi n° 43-20',
  'node_2': 'services de confiance pour les transactions électroniques',
  'edge': 'Rappelant',
  'chunk_id': '3b43557b46c442bfad59617aad10dbc5'},
 {'node_1': 'Loi n° 53-05',
  'node_2': 'échange électronique des données juridiques',
  'edge': 'Rappelant',
  'chunk_id': '3b43557b46c44

## Calculating contextual proximity

In [ ]:
import requests

# Helper function to query the LLM with a structured prompt in French
def get_relation(node_1: str, node_2: str, model_name: str, base_url: str) -> str:
    SYS_PROMPT = (
        "Votre tâche consiste à identifier la relation entre deux concepts dans le contexte du droit marocain. "
        "Si aucune relation pertinente n'est trouvée dans le droit marocain ou dans le contexte du Maroc, fournissez une relation basée sur vos connaissances générales. "
        "Formatez votre sortie sous forme d'une phrase simple et concise décrivant la relation entre les deux concepts. "
        "Assurez-vous que la relation est pertinente et claire pour un lecteur non expert."
    )
    prompt = (
        f"{SYS_PROMPT}\n"
        f"Concepts à analyser :\n"
        f"1. {node_1}\n"
        f"2. {node_2}\n"
        "Quelle est leur relation ?"
    )
    
    payload = {
        "model": model_name,
        "prompt": prompt,
    }
    
    try:
        response = requests.post(f"{base_url}/api/generate", json=payload)
        response.raise_for_status()
        result = response.json()
        # Extract the response or use a fallback if unavailable
        relationship = result.get("response", "").strip()
        return relationship if relationship else "contextual proximity"
    except requests.exceptions.RequestException as e:
        print(f"LLM request failed: {e}")
        return "contextual proximity"


In [7]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = dfg2.apply(
    lambda row: get_relation(row["node_1"], row["node_2"], "llama3.3", "https://88f9-54-158-13-109.ngrok-free.app/"), axis=1
  )
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge


### Merge both the dataframes

In [8]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,arrêté,"dématérialisation des procédures, documents et...",3b43557b46c442bfad59617aad10dbc5,Portée,4
1,arrêté du ministre délégué,"ministre de l'économie et des finances, chargé...",3b43557b46c442bfad59617aad10dbc5,Publié par,4
2,arrêté n° 1692-23,4 hija 1444 (23 juin 2023),3b43557b46c442bfad59617aad10dbc5,Date d'émission,4
3,bulletin officiel n° 7222,30 moharrem 1445 (17-8-2023),3b43557b46c442bfad59617aad10dbc5,Publié dans,4
4,dahir n° 1-56-211,garanties pécuniaires des soumissionnaires et ...,3b43557b46c442bfad59617aad10dbc5,Rappelant,4
5,loi n° 112-13,nantissement des marchés publics,3b43557b46c442bfad59617aad10dbc5,Rappelant,4
6,loi n° 43-20,services de confiance pour les transactions él...,3b43557b46c442bfad59617aad10dbc5,Rappelant,4
7,loi n° 53-05,échange électronique des données juridiques,3b43557b46c442bfad59617aad10dbc5,Rappelant,4
8,loi n° 55-19,simplification des procédures et formalités ad...,3b43557b46c442bfad59617aad10dbc5,Rappelant,4
9,loi n° 69-00,contrôle financier de l'état sur les entrepris...,3b43557b46c442bfad59617aad10dbc5,Rappelant,4


## Calculate the NetworkX Graph

In [9]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(20,)

In [10]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

### Calculate communities for coloring the nodes

In [11]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  12
[['30 moharrem 1445 (17-8-2023)', 'bulletin officiel n° 7222'], ['4 hija 1444 (23 juin 2023)', 'arrêté n° 1692-23'], ['arrêté'], ['arrêté du ministre délégué'], ["contrôle financier de l'état sur les entreprises publiques et autres organismes", 'loi n° 69-00'], ['dahir n° 1-56-211', 'garanties pécuniaires des soumissionnaires et adjudicataires de marchés publics'], ['dématérialisation des procédures, documents et pièces relatifs aux marchés publics'], ['loi n° 112-13', 'nantissement des marchés publics'], ['loi n° 43-20', 'services de confiance pour les transactions électroniques'], ['loi n° 53-05', 'échange électronique des données juridiques'], ['loi n° 55-19', 'simplification des procédures et formalités administratives'], ["ministre de l'économie et des finances, chargé du budget"]]


### Create a dataframe for community colors

In [12]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,30 moharrem 1445 (17-8-2023),#dba157,1
1,bulletin officiel n° 7222,#dba157,1
2,4 hija 1444 (23 juin 2023),#db5791,2
3,arrêté n° 1692-23,#db5791,2
4,arrêté,#db5f57,3
5,arrêté du ministre délégué,#5f57db,4
6,contrôle financier de l'état sur les entrepris...,#57dba1,5
7,loi n° 69-00,#57dba1,5
8,dahir n° 1-56-211,#5791db,6
9,garanties pécuniaires des soumissionnaires et ...,#5791db,6


### Add colors to the graph

In [13]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [14]:
from pyvis.network import Network

graph_output_directory = "./docs/index_llama3.3.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./docs/index_qwen.html
